# Find mismatches

Source files are created from the APPLAUSE database by this query:

```
SELECT * 
FROM applause_dr4.source 
WHERE plate_id = 19019
ORDER BY source_id
```
with another separate call for the source_calib table.

We look for sources that show up in two separate plates. The ones that don't are the candidates we should look in more detail.

The script exploits parallelism to expedite the search.

In [1]:
import os, io
import csv
from math import sqrt

import multiprocessing as mp
from multiprocessing import Pool

import numpy as np

from astropy import units as u
from astropy.io import fits
from astropy.io.fits import Header, Card
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Longitude, Latitude

from regions import PolygonSkyRegion
from mocpy import MOC

from settings import get_parameters, fname, current_dataset
from library import Worker, Worker2, is_in_jupyter

In [2]:
in_jupyter = is_in_jupyter()

print(in_jupyter)

True


In [3]:
par = get_parameters(current_dataset)
par

{'nproc': 8,
 'sextractor_flags': 4,
 'model_prediction': 0.8,
 'max_flux_threshold': 0.3,
 'elongation': 1.5,
 'annular_bin': 7,
 'flag_rim': 0,
 'min_acceptable_flux': 25000,
 'min_fwhm': 5.0,
 'max_fwhm': 7.5,
 'qfit_max': 5.0,
 'cfit_max': 5.0,
 'invert_east': [False, False],
 'invert_north': [False, False],
 'table1': 'sources_9319.csv',
 'table2': 'sources_9320.csv',
 'table1_calib': 'sources_calib_9319.csv',
 'table2_calib': 'sources_calib_9320.csv',
 'table_matched': 'table_match_9319_9320.fits',
 'table_non_matched': 'table_nomatch_9319_9320.fits',
 'table_psf_nonmatched': 'table_psf_nomatch_9319_9320.fits',
 'image1': 'GS00768_x.fits',
 'image2': 'GS00769_x.fits'}

## Read source tables generated by APPLAUSE database

From both **applause_dr4.source** and **applause_dr4.source_calib**. We need both, but we cannot get a joined version directly from APPLAUSE. Maybe they are limiting the output size. In any way, we get both and join them locally here.

In [4]:
table_src_1 = Table.read(fname(par['table1']), format='ascii.csv')
table_src_2 = Table.read(fname(par['table2']), format='ascii.csv')

table_calib_1 = Table.read(fname(par['table1_calib']), format='ascii.csv')
table_calib_2 = Table.read(fname(par['table2_calib']), format='ascii.csv')

In [5]:
table_1 = join(table_calib_1, table_src_1, keys='source_id')
table_2 = join(table_calib_2, table_src_2, keys='source_id')

In [6]:
print(len(table_1), len(table_2))

151083 149379


In [7]:
table_1

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349380000001,34938,12209,9319,102,0,9,5639.7905,3.401241,16,1.7833301e-32,0.9547166195413297,55.595146543415524,0.0968428,0.0834128,116.19279638684111,-6.652284001846378,30.80659271017268,48.87573004153747,0.5649584552501794,0.009414744970332244,0.82506563764253,47848,765572,13.764889,17.019691,1.0456479,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(0.0166629484345062 , 0.970318355311346)",34938,12209,9319,102,1,1981.2966,2.09694866605748,5.599326,0.4896937,-0.21710949,0.25392026,0.022170555,-0.22048235,11.434343,1981,3,0,1981.2966,3.401241,0.25392026,0.022170555,-0.22048235,NULL,NULL,NULL,NULL,NULL,8.912098,0.015099465,2723709.5,37869.758,11.8073635,0.04905693,189258.14,8549.192,5220.535,11.926231,37,6.0827627,-2946.5583,16,5639.7905,3.401241,9,1,0,0,1.7833301e-32,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380000002,34938,12209,9319,102,0,9,4418.4404,3.258278,17,7.2918836e-32,359.83372973563047,55.61027902578035,0.0968428,0.0834128,115.5703005451683,-6.514978507996677,30.05245588402188,49.27228790327279,0.5648165883464343,-0.0016390817593808333,0.8252148416865032,226685,3626974,7.1868157,17.647688,1.0492189,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.28028334361818 , 0.970582466952612)",34938,12209,9319,102,2,4482.26,1.9539996824477597,7.8481503,0.43700832,0.17823699,0.33551568,0.018726936,0.17961313,17.958813,4483,3,0,4482.26,3.258278,0.33551568,0.018726936,0.17961313,NULL,NULL,NULL,NULL,NULL,8.543116,0.011961347,3826077.8,42140.926,11.676017,0.04631115,213596.0,9108.542,5166.785,18.241182,42,6.4807405,-3785.3176,17,4418.4404,3.258278,9,1,0,0,7.2918836e-32,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380000003,34938,12209,9319,102,0,9,5602.6445,11.423781,0,0.50449014,0.9335739495525104,55.59754913562532,0.0968428,0.0834128,116.18142452652452,-6.647712902829138,30.794334523762917,48.884868540847926,0.564927297412249,0.009205705749936007,0.8250893306910094,47848,765569,31.244658,17.031

## Remove undesirable sources

Use a variety of criteria from SEXTRACTOR to get rid of possible contaminants.

We want to clean the first table only, since only well-defined star images from the first image should be present in our sample. The second table is taken as is; we want to see everything that is near the sources selected from the firsr table. 

In [8]:
table_1copy = table_1.copy()
table_2copy = table_2.copy()

# Extraction flags 
mask = table_1copy['sextractor_flags_1'] <= par['sextractor_flags']
table_1copy = table_1copy[mask]

# Likelihood of being valid star image 
mask = table_1copy['model_prediction_1'] > par['model_prediction']
table_1copy = table_1copy[mask]

# Round objects 
mask = table_1copy['elongation'] < par['elongation']
table_1copy = table_1copy[mask]

# Ring
mask = table_1copy['annular_bin_1'] <= par['annular_bin']
table_1copy = table_1copy[mask]

# Is source at plate rim ?
mask = table_1copy['flag_rim'] == par['flag_rim']
table_1copy = table_1copy[mask]

# flux threshold - set it to fraction of max peak in image
flux_threshold = max(table_1copy['flux_max']) * par['max_flux_threshold']
mask = table_1copy['flux_max'] > flux_threshold
table_1copy = table_1copy[mask]

print(len(table_1copy), len(table_2copy))

37165 149379


## Remove scanner artifacts

When plates were scanned twice or more, usually in two directions perpendicular to each other, all scans are included in a single table. Stars and artifacts *on* the plate will share the same celestial coordinates in both scans, but scanner-induced artifacts won't, because they move in x,y when the plate is rotated 90 deg. Thus we remove all rows that have no duplicates.  

To expedite the search, we split the job among all available performance CPUs.

In [9]:
# number of processors
nproc = par['nproc']

In [10]:
# The matching code lives in file library.py - it has to be kept in a separate 
# file because of restrictions in name space imposed by the parallelization library.

# Here, the parallelized code stores its product, the list of indices that have duplicates.
matched = []

# callback function to collect results from parallel workers
def collect_result(results):
    matched.extend(results)
    
results = []
pool = Pool(nproc)

row_range = int(len(table_1copy) / nproc)

for p in range(nproc):
    # workers are defined over ranges of rows in the table_1copy table
    worker = Worker2("w"+str(p), table_1copy, int(p*row_range), int((p+1)*row_range))

    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)

for r in results:
    r.wait()

pool.close()

Worker  w0  -  0 4645
Worker  w1  -  4645 9290
Worker  w2  -  9290 13935
Worker  w3  -  13935 18580
Worker  w4  -  18580 23225
Worker  w5  -  23225 27870
Worker  w6  -  27870 32515
Worker  w7  -  32515 37160
w0  -  0% .  0 28125 40349380002595     40349390035184    0.047783726313355146 0.026958373913998912
w3  -  1% .  14000 33265 40349380049893     40349390049325    0.1516738321242883 0.18036166155468436
w2  -  4% .  9500 21649 40349380036233     40349390017328    0.09872771006484982 0.2292194520208568
w3  -  12% .  14500 31898 40349380051640     40349390045457    0.05095113926927297 0.059765815004197975
w1  -  7% .  5000 35101 40349380022296     40349390054899    0.3836681372604289 0.13266166589573913
w2  -  15% .  10000 21209 40349380037786     40349390016096    0.10350143884352292 0.07366236015400318
w0  -  10% .  500 24800 40349380005817     40349390026189    0.22295340163509536 0.01146578455859526
w3  -  22% .  15000 31446 40349380053422     40349390044198    0.052121958333373186

In [11]:
print(len(matched), " rows have duplicates")

16972  rows have duplicates


In [12]:
# if a small number of duplicates, or none, was detected, ignore them.
if len(matched) > len(table_1copy) / 10:

    # lots of duplicates: keep only the duplicated rows
    table_1copy = table_1copy[matched]

print(len(table_1copy))

16972


## Look for matches between 1st and 2nd tables

In [13]:
# Same code as above (can't be put in a function to avoid duplication).
#
# It generates the list of indices (in the first table) of all objects 
# for which it could find at least one matching entry in the second table.
matched = []

results = []
pool = Pool(nproc)

row_range = int(len(table_1copy) / nproc)

for p in range(nproc):
    # workers are defined over ranges of rows in the table_1copy table
    worker = Worker("w"+str(p), table_1copy, table_2copy, int(p*row_range), int((p+1)*row_range))

    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)

for r in results:
    r.wait()

pool.close()

Worker  w0  -  0 2121
Worker  w1  -  2121 4242
Worker  w2  -  4242 6363
Worker  w3  -  6363 8484
Worker  w4  -  8484 10605
Worker  w5  -  10605 12726
Worker  w6  -  12726 14847
Worker  w7  -  14847 16968
w0  -  0% .  0 49126 40349380049667     40349400049127    0.028607872627617326 0.07284718661253464
w4  -  0% .  8500 21390 40349380021848     40349400021391    0.09387439085912774 0.13621486072850075
w6  -  12% .  13000 5031 40349380005263     40349400005032    0.4671856823961207 0.11492211984602818
w7  -  7% .  15000 13939 40349380014218     40349400013940    0.4122819668509692 0.27282305167659615
w3  -  6% .  6500 43043 40349380043694     40349400043044    0.08448045350633038 0.04650121667566509
w6  -  36% .  13500 7787 40349380008084     40349400007788    0.003658671062112262 0.03557458821603632
w2  -  12% .  4500 36493 40349380036989     40349400036494    0.2887727949882901 0.19013409250874247
w7  -  30% .  15500 15735 40349380016090     40349400015736    0.12184600807358947 0.0154

In [14]:
print(len(matched), " sources detected in 1st plate with a match in 2nd plate")

16895  sources detected in 1st plate with a match in 2nd plate


## Get the non-matched objects

Remove the matched rows from the first table. Whatever remains, is the table of objects for which a match couldn't be found.

Remove duplicated indices first, and save original table since removal is done in-place.

In [15]:
table_3 = table_1copy.copy()

m = list(dict.fromkeys(matched))
table_3.remove_rows(m)

In [16]:
table_3

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349380049749,34938,12209,9319,102,0,1,1250.612,3335.4937,0,1.0,359.7004097124922,56.91710661093143,0.0968428,0.0834128,115.76366476838677,-5.220265643296449,31.21061615930761,50.34699064777934,0.5458443597997983,-0.0028541571549151688,0.837881727131988,226813,3629009,13.722612,17.618925,1.049052,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.27795647025558 , 0.993390911069384)",34938,12209,9319,102,49749,4839.25,5172.498264939447,1.8430362,1.5482224,-36.09808,0.039377335,0.033299033,-33.30857,1.1904207,4839,5172,0,4839.25,5172.5063,0.039377335,0.033299033,-33.30857,NULL,NULL,NULL,NULL,NULL,10.129967,0.031442277,887183.1,25686.057,10.598527,0.01815991,576221.3,9635.476,27906.65,2.8575315,37,6.0827627,896.25507,0,1250.612,3335.4937,1,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380050009,34938,12209,9319,102,0,3,2845.9915,3002.8276,0,1.0,0.5524371329881319,56.91388265377987,0.0968428,0.0834128,116.22094516463082,-5.3150472719758834,31.776810871096725,50.05542912446628,0.5458735927920557,0.0052633927031881505,0.8378510114510485,47868,765902,12.321559,17.154264,1.0464005,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(0.00964184688092068 , 0.993334642402147)",34938,12209,9319,102,50009,3002.8276,5197.134402069652,2.0523188,1.7397393,-38.93845,0.035164814,0.030379647,-44.994102,1.1796703,3003,5197,0,3002.8276,5197.142,0.035164814,0.030379647,-44.994102,NULL,NULL,NULL,NULL,NULL,10.151687,0.024002116,869611.56,19219.629,10.269172,0.015022721,780425.5,10795.686,28447.234,2.538284,53,7.28011,2399.8057,0,2845.9915,3002.8276,3,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380050883,34938,12209,9319,102,0,2,2334.2837,3210.6587,0,1.0,0.27650032434854666,56.94287941436541,0.0968428,0.0834128,116.07816761345107,-5.2575907207544,31.62066594142601,50.17157691972454,0.5454685186770176,0.002632364799793251,0.838127177573822,47870,765922,21.231281,17.303795,1.0472448,NULL,NULL,NULL,NULL,NULL,N

## Write result to FITS table

In [17]:
table_3.write(fname(par['table_non_matched']), format='fits', overwrite=True)

In [18]:
# table with the matched objects only (selected by index array m)

table_1copy[m].write(fname(par['table_matched']), format='fits', overwrite=True)

In [19]:
table_1copy[m]

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349380003507,34938,12209,9319,102,1,7,4561.7173,529.94025,0,0.99972034,0.47970994209299533,55.73569079884082,0.0968428,0.0834128,115.9546051765027,-6.463774436301726,30.611719370183998,49.14996638421357,0.5629916117085119,0.004713767515459602,0.8264491669435347,47850,765601,30.833323,17.272127,1.0470653,13.571841,0.5161281,13.42579,0.51613533,12.604428,0.51613533,13.78388,-0.21203943,-0.20398235,0,1,1.1778167,13.367859,0.6390662,420893605173226240,NULL,0.821361,0.27246258,1,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(0.00837251794407411 , 0.972771315313281)",34938,12209,9319,102,3507,3053.49,529.0668227947147,4.4319735,4.234598,59.497673,0.017291162,0.016104054,63.07245,1.0466101,3053,528,0,3053.49,529.94025,0.017291162,0.016104054,63.07245,NULL,NULL,NULL,NULL,NULL,7.716681,0.0050314157,8190824.5,37947.92,7.7206254,0.0034333582,8161122.0,25801.156,43485.26,5.6977706,325,18.027756,3578.5408,0,4561.7173,529.94025,7,0,0,1,0.99972034,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380003522,34938,12209,9319,102,1,5,3710.108,547.52466,0,1.0,359.36810892407624,55.75091495977316,0.0968428,0.0834128,115.34082690593699,-6.323420023706918,29.86527408334801,49.54408230812399,0.5627575020499181,-0.0062066676532761145,0.8265987364877702,226772,3628364,14.398606,17.89281,1.0506535,15.151942,0.4970464,14.945799,0.49706137,13.786499,0.49706137,15.064298,0.08764504,0.100631714,0,1,1.1778167,15.153673,0.6390662,1994714654883115904,NULL,1.1593,0.43851635,2,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.27215672739075 , 0.973037027047404)",34938,12209,9319,102,3522,5523.8604,546.686243225803,3.1585107,2.8655984,-89.50723,0.024879092,0.022356646,86.3292,1.1022167,5524,547,0,5523.8604,547.52466,0.024879092,0.022356646,86.3292,NULL,NULL,NULL,NULL,NULL,8.776608,0.009254045,3085719.8,26294.096,8.855179,0.0067264424,2870308.5,17778.053,35598.785,4.085698,149,12.206555,4244.185,0,3710.108,547.52466,5,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380003543,34938,12209,9319,102,1,6,4022.6018,555.14484,0,1.0,0.00600245433810187,55.74787441643706,0.0968428,0.0834128,115.6938391